In [1]:
! pip install torchsummary
import sys
from torchsummary import summary
import h5py
import numpy as np
import megengine.data as data
import megengine.data.transform as T
# import megengine
import cv2
import os
from megengine.data.dataset import Dataset
import random
import math
import scipy
from scipy import io

# import megengine as mge
# import megengine.module as M
# import megengine.functional as F

# from megengine.data.transform import ToMode
# from megengine.data import DataLoader, RandomSampler

import torch
import torchvision
import torch.utils.data as Data
# import torch.nn as M
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib import cm

from tqdm.notebook import tqdm
from torch.autograd import Variable
import wandb
wandb.login()

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sakura_1986 (use `wandb login --relogin` to force relogin)


True

# Configs

In [2]:
class cfg:
    model = 'Predictor_l'
    dataset = 'MEG'
    seed = 42
    Meg_train_length = 7000    #0-8192
    folder = "/root/autodl-tmp/SIDD_Small_Raw_Only/"
    bs = 64 # batch size
    cut_size = 256
    device = 'cuda:0'
    load_pretrain = False
    lr = 1e-3
    n_steps = 10
    gamma = 0.7
    start_epoch = 0
    n_epochs = 200
    model_name = model+'_'+dataset+'_'+"nkh_train00"

# Utils

In [3]:
def psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
#     mse = torch.mean((img1 - img2) ** 2)
    return 20 * torch.log10(1.0 / torch.sqrt(mse)).item()

In [4]:
def toTensor(img):
    img = torch.from_numpy(img)
    return img.float()

In [5]:
def add_str(cls):
    def __str__(self):
        return str(self.__dict__)
    cls.__str__ = __str__
    return cls

In [6]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
        
setup_seed(cfg.seed)

In [7]:
def mean(list1):
    return sum(list1)/len(list1)

def compute_score(igt, ipred):
    pred_name, gt_name = sys.argv[1:3]
    samples_gt = igt
    samples_pred = ipred
    means = samples_gt.mean(axis=(1, 2))
    weight = (1/means)**0.5
    diff = torch.abs(samples_pred - samples_gt).mean(axis=(1, 2))
    diff = diff * weight
    score = diff.mean()
    return score

# Dataloader

In [8]:
class MEGDataset(Dataset):
    def __init__(self, data_path):
        self.input_path = data_path +  '/competition_train_input.0.2.bin'
        self.gt_path = data_path + '/competition_train_gt.0.2.bin'
        self.content = open(self.input_path, 'rb').read()
        self.input_images = np.frombuffer(self.content, dtype = 'uint16').reshape((-1,256,256))
        self.content = open(self.gt_path, 'rb').read()
        self.gt_images = np.frombuffer(self.content, dtype = 'uint16').reshape((-1,256,256))
        print(len(self.input_images))
        
    
    def __getitem__(self, idx):
        inoisy = np.float32([self.input_images[idx, :, :]]) * np.float32(1 / 65536)
        igt = np.float32([self.gt_images[idx, :, :]]) * np.float32(1 / 65536)
        return toTensor(inoisy), toTensor(igt)
    
    def __len__(self):
        return len(self.input_images)

In [9]:
class SIDDDataset(Dataset):
    def __init__(self, mode, data_list):
        self.image_folder = cfg.folder + "new_Data/"
        self.image_list_all = os.listdir(self.image_folder)
        if mode == 1:  #train or val
            data_list_str = ' '.join(data_list)  #全部训练集源文件名组成的字符串
            self.image_list = [img_idx for img_idx in self.image_list_all if img_idx.rpartition('_')[0] in data_list_str]
        else:
            self.image_list = self.image_list_all
        self.cut = cfg.cut_size

    # get the sample
    def __getitem__(self, idx):
        # get the index
        image_id = self.image_list[idx]
        inoisy = scipy.io.loadmat(self.image_folder + self.image_list[idx] + "/GT_RAW_010.MAT")
        igt = scipy.io.loadmat(self.image_folder + self.image_list[idx] + "/NOISY_RAW_010.MAT")
        inoisy = np.float32([np.array(inoisy['var_name'])])
        igt = np.float32([np.array(igt['var_name'])])
        return toTensor(inoisy), toTensor(igt)

    def __len__(self):
        return len(self.image_list)

In [10]:
from torch.utils.data import random_split
# a = os.listdir("/root/autodl-tmp/SIDD_Small_Raw_Only/new_Data/")
# len(a)

In [11]:
if cfg.dataset=="MEG":
    Meg_data_path = '/root/data/burst_raw/'
    dataset = MEGDataset(Meg_data_path)
    train_set, vali_set = random_split(dataset=dataset, lengths=[cfg.Meg_train_length, 8192-cfg.Meg_train_length])
elif cfg.dataset == 'SIDD':
    random.seed(32)
    ori_list = os.listdir("/root/autodl-tmp/SIDD_Small_Raw_Only/Data/")
    random.shuffle(ori_list)
    train_list = ori_list[:138]    #0-160，需要根据val_list中的数据数设定计算psnr时的被除数
    val_list = ori_list[138:]
    train_set = SIDDDataset(1, train_list)
    vali_set = SIDDDataset(1, val_list)
    
else:
    print('set an existing dataset')

8192


In [12]:
train_dataloader = Data.DataLoader(train_set,
      shuffle=True, batch_size=cfg.bs
)
vali_dataloader = Data.DataLoader(vali_set,
      shuffle=False, batch_size=1
)

# Net

In [13]:
class Predictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(50, 4, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
    def forward(self, x):
        n, c, h, w = x.shape
        x = x.reshape((n, c, h // 2, 2, w // 2, 2)).permute(0, 1, 3, 5, 2, 4).reshape((n, c * 4, h // 2, w // 2))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.reshape((n, c, 2, 2, h // 2, w // 2)).permute(0, 1, 4, 2, 5, 3).reshape((n, c, h, w))
        return x

In [14]:
class Predictor_l(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 100, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(100, 200, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(200, 200, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(200, 200, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(200, 100, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(100, 4, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
    def forward(self, x):
        n, c, h, w = x.shape
        x = x.reshape((n, c, h // 2, 2, w // 2, 2)).permute(0, 1, 3, 5, 2, 4).reshape((n, c * 4, h // 2, w // 2))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.reshape((n, c, 2, 2, h // 2, w // 2)).permute(0, 1, 4, 2, 5, 3).reshape((n, c, h, w))
        return x

In [15]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

In [16]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 8, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
sam = SpatialAttention()
total = sum([param.nelement() for param in sam.parameters()])
print('  Number of params: %.3fk' % (total / 1e3))

  Number of params: 0.784k


In [17]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(4, 8, 3, stride=3, padding=1),  # (b, 16, 10, 10)
            nn.LeakyReLU(True),
            nn.MaxPool2d(2, stride=2),  # (b, 16, 5, 5)
            nn.Conv2d(8, 16, 3, stride=2, padding=1),  # (b, 8, 3, 3)
            nn.LeakyReLU(True),
            nn.MaxPool2d(2, stride=1),  # (b, 8, 2, 2)
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # (b, 8, 3, 3)
            nn.LeakyReLU(True),
            nn.MaxPool2d(2, stride=1),  # (b, 8, 2, 2)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 32, 4, stride=2),  # (b, 16, 5, 5)
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(32, 16, 5, stride=2, padding=1),  # (b, 8, 15, 15)
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=2, padding=1),  # (b, 1, 28, 28)
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(8, 4, 4, stride=3, padding=1),
            nn.Tanh()
        )
#         self.cam = ChannelAttention(self.inplanes)
        self.sam = SpatialAttention()
    def forward(self, x):
        n, c, h, w = x.shape
        x = x.reshape((n, c, h // 2, 2, w // 2, 2)).permute(0, 1, 3, 5, 2, 4).reshape((n, c * 4, h // 2, w // 2))
#         print(f'x.shape: {x.shape}')
        x = self.encoder(x)
#         print(f'x.shape: {x.shape}')
        x=self.sam(x)
#         print(f'x.shape: {x.shape}')
        x = self.decoder(x)
#         print(f'x.shape: {x.shape}')
#         x=self.sam(x)
        x = x.reshape((n, c, 2, 2, h // 2, w // 2)).permute(0, 1, 4, 2, 5, 3).reshape((n, c, h, w))
        return x
        


# Debugging

In [21]:
net = Predictor_l()
total = sum([param.nelement() for param in net.parameters()])
print('  Number of params: %.3fk' % (total / 1e3))

  Number of params: 1088.004k


In [22]:
# 调试Autoencoder输出维度
x = torch.randn(1,1,256,256)
y = net(x)
print(f'x.shape: {x.shape}')
print(f'y.shape: {y.shape}')
summary(net,(1,256,256),batch_size=1,device='cpu')

x.shape: torch.Size([1, 1, 256, 256])
y.shape: torch.Size([1, 1, 256, 256])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [1, 100, 128, 128]           3,700
         LeakyReLU-2         [1, 100, 128, 128]               0
            Conv2d-3         [1, 200, 128, 128]         180,200
         LeakyReLU-4         [1, 200, 128, 128]               0
            Conv2d-5         [1, 200, 128, 128]         360,200
         LeakyReLU-6         [1, 200, 128, 128]               0
            Conv2d-7         [1, 200, 128, 128]         360,200
         LeakyReLU-8         [1, 200, 128, 128]               0
            Conv2d-9         [1, 100, 128, 128]         180,100
        LeakyReLU-10         [1, 100, 128, 128]               0
           Conv2d-11           [1, 4, 128, 128]           3,604
        LeakyReLU-12           [1, 4, 128, 128]               0
Total params: 1,088,004
Tra

# Train

In [25]:
from tqdm import tqdm
def train(train_loader, vali_loader):
    if cfg.model == 'Predictor':
        model = Predictor().to(cfg.device)
    elif cfg.model == 'Autoencoder':
        model=Autoencoder().to(cfg.device)
    elif cfg.model == 'Predictor_l':
        model = Predictor_l().to(cfg.device)
    
    # 加载预训练权重
    if cfg.load_pretrain:
        # TODO
        s = 0

    criterion_L1 = torch.nn.L1Loss().to(cfg.device)
    optimizer = torch.optim.Adam([paras for paras in model.parameters() if paras.requires_grad == True], lr=cfg.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=cfg.n_steps, gamma=cfg.gamma)

    loss_list = []
    
    # 开始训练
    for idx_epoch in range(cfg.start_epoch, cfg.n_epochs):
        # record loss
        loss_epoch = []
        wandb.log({"epoch": idx_epoch})
        for idx_iter, (inoisy, igt) in tqdm(enumerate(train_loader)):
            inoisy, igt = Variable(inoisy).to(cfg.device), Variable(igt).to(cfg.device)
#             print(f'igt.shape: {igt.shape}')
            ipred = model(inoisy)
#             print(len(ipred))
            loss = criterion_L1(ipred, igt)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_epoch.append(loss.data.cpu())
            
        scheduler.step()
        lr_now = scheduler.get_last_lr()
        wandb.log({"lr": float(lr_now[-1])})
        
        # 输出结果
        if idx_epoch % 1 == 0:
            print('Epoch--%4d, loss--%f' %
                  (idx_epoch + 1, float(np.array(loss_epoch).mean())))
            wandb.log({"loss": float(np.array(loss_epoch).mean())})
            
        # 保存模型
        if idx_epoch % 5 == 0:
            torch.save({'epoch': idx_epoch + 1, 'state_dict': model.state_dict()},
                       'log/' + cfg.model_name + '_' + 'epoch' + str(idx_epoch + 1) + '.pth.tar')
            
        # validation
        if idx_epoch % 5 == 0:
            psnr_list = []
            score_list = []
            for idx_iter, (inoisy, igt) in tqdm(enumerate(vali_loader)):
                inoisy, igt = Variable(inoisy).to(cfg.device), Variable(igt).to(cfg.device)
                with torch.no_grad():
                    ipred = model(inoisy)
                s = compute_score(igt, ipred)
                p = psnr(inoisy, ipred)
                psnr_list.append(p)
                score_list.append(s)
            val_len = 8192 - cfg.Meg_train_length
            score = 5*math.log(100/mean(score_list),10)
            print("Tested PSNR", str(sum(psnr_list) / val_len))
            print("Tested SCORE", str(score))
            wandb.log({"PSNR": float(sum(psnr_list) / val_len)})
            wandb.log({"SCORE": float(score)})

In [ ]:
train_cfg = {
                "batch size": cfg.bs,
                "cut_size":cfg.cut_size,
                "num_epoch":cfg.n_epochs,
                "init_lr":cfg.lr,
                "n_steps":cfg.n_steps,
                "change_gamma":cfg.gamma,
                "Model Name":cfg.model_name
}

run_name = cfg.model_name

wandb.init(project="blind-denoising",
           config = train_cfg,
           entity="hanhan_tie",
#            entity="sakura_1986",
           name = run_name)

wandb.run.save()

train(train_dataloader, vali_dataloader)

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
110it [04:24,  2.40s/it]


Epoch--   1, loss--0.016773


1192it [04:06,  4.83it/s]


Tested PSNR 37.321229833084466
Tested SCORE 17.257641838163263


110it [03:52,  2.11s/it]


...

110it [07:23,  4.03s/it]


Epoch--  25, loss--0.002746


110it [06:33,  3.58s/it]


Epoch--  26, loss--0.002638


504it [02:10,  3.17it/s]

#  Inference

In [18]:
net = Predictor_l().to('cuda:0')
model = torch.load('./log/Predictor_l_MEG_nkh_train00_epoch96.pth.tar')
net.load_state_dict(model['state_dict'])

<All keys matched successfully>

In [19]:
print('prediction')

content = open('/root/data/burst_raw/competition_test_input.0.2.bin', 'rb').read()
samples_ref = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
fout = open('/root/out2.bin', 'wb')

import tqdm

for i in tqdm.tqdm(range(0, len(samples_ref), 64)):
    i_end = min(i + 64, len(samples_ref))
    batch_inp = torch.tensor(np.float32(samples_ref[i:i_end, None, :, :]) * np.float32(1 / 65536)).to('cuda:0')
    pred = net(batch_inp)
#     pred = (pred.numpy()[:, 0, :, :] * 65536).clip(0, 65535).astype('uint16')
    pred = (pred.detach().cpu().numpy()[:, 0, :, :] * 65536).clip(0, 65535).astype('uint16')
    fout.write(pred.tobytes())

fout.close()

prediction


100%|██████████| 16/16 [00:05<00:00,  2.96it/s]


In [ ]:
# data = scipy.io.loadmat("/root/autodl-tmp/SIDD_Small_Raw_Only/" + "new_Data/" + "0001_001_S6_00100_00060_3200_L_0/NOISY_RAW_010.MAT")
# inoisy = np.float32([np.array(data['var_name'])])
# print(inoisy.shape)

In [ ]:
# inoisy = h5py.File("/root/autodl-tmp/SIDD_Small_Raw_Only/" + "Data/" + "0001_001_S6_00100_00060_3200_L/NOISY_RAW_010.MAT")
# inoisy = np.float32([np.array(inoisy['x'])])
# print(inoisy.shape)

In [ ]:
# class Autoencoder(M.Module):
#     def __init__(self):
#         super().__init__()
#         self.encoder = M.Sequential([
#             # Input(shape=(28, 28, 1,)),
#             M.Conv2D(4, 50, 3, padding = 1, bias = True),
#             M.MaxPooling2D((2, 2), padding='same'),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(3,3)),
#             M.MaxPooling2D((1, 1), padding='same'),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.MaxPooling2D((1, 1), padding='same'),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.MaxPooling2D((2, 2), padding='same'),
#         ])
#         self.decoder=M.Sequential([
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(2,2)),
#             M.UpSampling2D((2, 2)),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(3,3)),
#             M.UpSampling2D((2, 2)),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.UpSampling2D((1, 1)),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.UpSampling2D((1, 1)),
#             M.Conv2D(1, (3, 3), padding='same', activation='sigmoid')
#         ])
#     def forward(self, inputs):
#         e=self.encoder(inputs)
#         y=self.decoder(e)
#         return encoder,decoder

In [ ]:
# # 检查参数量
# model = Predictor()
# # print(model)
# autoencoder=Autoencoder()

In [ ]:
# from megengine.utils.module_stats import module_stats

# input_data = np.random.rand(1, 1, 256, 256).astype("float32")
# total_stats, stats_details = module_stats(
#     net,
#     inputs = (input_data,),
#     cal_params = True,
#     cal_flops = True,
#     logging_to_stdout = True,
# )

# print("params %.3fK MAC/pixel %.0f"%(total_stats.param_dims/1e3, total_stats.flops/input_data.shape[2]/input_data.shape[3]))